# Loading packages

In [1]:
import numpy as np
import pandas as pd
import glob

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
from pandas.api.types import CategoricalDtype

In [4]:
%load_ext RWinOut

In [5]:
%%R
library(gtools)
library(metafor)

# Loading data

### Odds ratio country by country

In [6]:
PT_contrasts_df2 = pd.read_excel("../output/contrasts_by_country.xlsx")

In [7]:
PT_contrasts_df2.head()

,Country,Effect,Item 1,Item 2,A1,B1,A2,B2,OR,p-value,OR2,Significance,Contrast,Sample Size,Significant,LogOR,Significance2,LogOR_difference,Replicates
0,Bulgaria,Certainty Effect,1,2,23,104,72,55,0.168937,2.425416e-10,5.919368,True,1 vs 2,127,Yes,-1.778230,1,-1.343796,Yes
1,Bulgaria,Certainty Effect,3,4,15,112,65,62,0.127747,1.080756e-11,7.827957,True,3 vs 4,127,Yes,-2.057702,1,0.040780,Yes
2,Bulgaria,Certainty Effect,7,8,109,18,76,51,4.063596,4.697618e-06,4.063596,True,7 vs 8,127,Yes,1.402068,1,-1.302852,Yes
3,Bulgaria,Reflection Effect,3,7,15,112,109,18,0.022117,5.761813e-35,45.214815,True,3 vs 7,127,Yes,-3.811425,1,0.038664,Yes
4,Bulgaria,Reflection Effect,4,8,65,62,76,51,0.703523,2.066420e-01,1.421418,False,4 vs 8,127,No,-0.351655,0,-1.300735,No


# Run meta-analytic models

In [8]:
%R -i PT_contrasts_df2

C:\ProgramData\Anaconda3\lib\site-packages\rpy2\robjects\pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [9]:
%%R
model_list = list()
estimate_list = list()
lb_list = list()
ub_list = list()
z_list = list()
p_list = list()
I2_list = list()
q_list = list()
qp_list = list()
X = 0
for (contrast in unique(PT_contrasts_df2$Contrast))
{
X = X + 1
ai = PT_contrasts_df2[PT_contrasts_df2["Contrast"]==contrast, "A1"]
bi = PT_contrasts_df2[PT_contrasts_df2["Contrast"]==contrast, "A2"]
ci = PT_contrasts_df2[PT_contrasts_df2["Contrast"]==contrast, "B1"]
di = PT_contrasts_df2[PT_contrasts_df2["Contrast"]==contrast, "B2"]
fit = rma.glmm(ai, bi, ci, di, measure="OR", method="ML", model="UM.RS")
print(contrast)
print(summary(fit))
model_list[[X]] = fit
estimate_list[[X]] = fit$beta[[1]]
lb_list[[X]] = fit$ci.lb
ub_list[[X]] = fit$ci.ub
z_list[[X]] = fit$zval
p_list[[X]] = fit$pval
I2_list[[X]] = fit$I2
q_list[[X]] = fit$QE.LRT[[1]]
qp_list[[X]] = fit$QEp.LRT

}



[1] "1 vs 2"

Random-Effects Model (k = 19; tau^2 estimator: ML)
Model Type: Unconditional Model with Random Study Effects

   logLik   deviance        AIC        BIC       AICc 
-132.6400    36.0190   273.2800   279.8304   274.4922   

tau^2 (estimated amount of total heterogeneity): 0.0469
tau (square root of estimated tau^2 value):      0.2166
I^2 (total heterogeneity / total variability):   50.4988%
H^2 (total variability / sampling variability):  2.0202

sigma^2 (estimated amount of study level variability): 0.0020
sigma (square root of estimated sigma^2 value):        0.0444

Tests for Heterogeneity:
Wld(df = 18) = 42.9385, p-val = 0.0008
LRT(df = 18) = 43.4788, p-val = 0.0007

Model Results:

estimate      se      zval    pval    ci.lb    ci.ub 
 -1.5810  0.0712  -22.2179  <.0001  -1.7205  -1.4415  *** 

---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

[1] "3 vs 4"

Random-Effects Model (k = 19; tau^2 estimator: ML)
Model Type: Unconditional Model with Random 

### Importing results from heterogeneity analysis to Python

In [10]:
%%R
z_list = as.numeric(unlist(z_list))
p_list = as.numeric(unlist(p_list))
I2_list = as.numeric(unlist(I2_list))
q_list = as.numeric(unlist(q_list))
qp_list = as.numeric(unlist(qp_list))

In [11]:
%R -o z_list
%R -o p_list
%R -o I2_list
%R -o q_list
%R -o qp_list
z_list = np.array(z_list)
p_list = np.array(p_list)
I2_list = np.array(I2_list)
q_list = np.array(q_list)
qp_list = np.array(qp_list)

### Structuring heterogeniety results into dataframe

In [18]:
het_df = pd.DataFrame([[str(a) + " vs " + str(b) for a, b in zip(PT_contrasts_df2.loc[PT_contrasts_df2["Country"]=="Sweden",
                                                    "Item 1"], PT_contrasts_df2.loc[PT_contrasts_df2["Country"]=="Sweden", "Item 2"])],
                       q_list, qp_list, I2_list, z_list, p_list]).transpose()
het_df.columns = ["Contrasts", "Q (df = 18)", "Q p-value", "I2", "z-statistic", "z p-value"]
het_df["Q (df = 18)"] = het_df["Q (df = 18)"].astype(float).round(2)
#het_df["Q p-value"] = het_df["Q p-value"].round(4)
het_df["I2"] = het_df["I2"].astype(float).round(0).astype(int)
het_df["z-statistic"] = het_df["z-statistic"].astype(float).round(2)
#het_df["z p-value"] = het_df["z p-value"].round(4)
het_df.head()
het_df.to_excel("../output/Meta analysis logodds heterogeneity.xlsx", index=False)
het_df.to_csv("../output/Meta analysis logodds heterogeneity.csv", index=False)

In [19]:
het_df

,Contrasts,Q (df = 18),Q p-value,I2,z-statistic,z p-value
0,1 vs 2,43.48,0.000684494,50,-22.22,2.30572e-109
1,3 vs 4,21.38,0.260489,0,-34.50,7.99424e-261
2,7 vs 8,9.86,0.936261,0,26.84,1.05263e-158
3,3 vs 7,56.59,7.19013e-06,72,-28.05,3.56816e-173
4,4 vs 8,35.13,0.00910908,73,0.30,0.764276
5,5 vs 9,121.20,2.49567e-17,83,-22.13,1.54052e-108
6,6 vs 10,23.99,0.155504,55,13.33,1.63637e-40
7,16 vs 17,77.47,2.36613e-09,85,4.89,9.8492e-07
8,4 vs 11,19.39,0.368077,0,31.72,9.24893e-221
9,5 vs 6,69.86,4.77705e-08,42,-33.78,4.00268e-250


### Importing log-odds results to Python

In [14]:
%%R
lb_list = as.numeric(unlist(lb_list))
ub_list = as.numeric(unlist(ub_list))
estimate_list = as.numeric(unlist(estimate_list))
p_list = as.numeric(unlist(p_list))

In [15]:
%R -o lb_list
%R -o ub_list
%R -o estimate_list
%R -o p_list
lb_list = np.array(lb_list)
ub_list = np.array(ub_list)
estimate_list = np.array(estimate_list)
p_list = np.array(p_list)

### Structuring results into dataframe

In [16]:
lodds_df = pd.DataFrame([estimate_list, lb_list, ub_list, p_list]).transpose()
lodds_df.columns = ["lodds", "lb", "ub", "p-value"]
lodds_df.head()

,lodds,lb,ub,p-value
0,-1.581017,-1.720487,-1.441547,2.305722e-109
1,-1.969901,-2.081812,-1.857990,7.994243e-261
2,1.332841,1.235518,1.430164,1.052629e-158
3,-3.332428,-3.565245,-3.099612,3.568163e-173
4,0.026007,-0.143975,0.195989,7.642756e-01


In [20]:
lodds_df["Contrasts"] = [str(a) + " vs " + str(b) for a, b in zip(PT_contrasts_df2.loc[PT_contrasts_df2["Country"]=="Sweden",
                                                    "Item 1"], PT_contrasts_df2.loc[PT_contrasts_df2["Country"]=="Sweden", "Item 2"])]

In [21]:
lodds_df["Succesful Replication"] = "Yes"
lodds_df.loc[lodds_df["p-value"]>.05, "Succesful Replication"] = "No"

In [22]:
lodds_df

,lodds,lb,ub,p-value,Contrasts,Succesful Replication
0,-1.581017,-1.720487,-1.441547,2.305722e-109,1 vs 2,Yes
1,-1.969901,-2.081812,-1.857990,7.994243e-261,3 vs 4,Yes
2,1.332841,1.235518,1.430164,1.052629e-158,7 vs 8,Yes
3,-3.332428,-3.565245,-3.099612,3.568163e-173,3 vs 7,Yes
4,0.026007,-0.143975,0.195989,7.642756e-01,4 vs 8,No
5,-3.532676,-3.845517,-3.219836,1.540525e-108,5 vs 9,Yes
6,0.918727,0.783602,1.053853,1.636367e-40,6 vs 10,Yes
7,0.582853,0.349460,0.816246,9.849195e-07,16 vs 17,Yes
8,1.703716,1.598432,1.809000,9.248932e-221,4 vs 11,Yes
9,-2.782012,-2.943432,-2.620591,4.002685e-250,5 vs 6,Yes


# Compute the difference in log-odds between the original results and the meta-analytic results transformed so that larger effects are positive and smaller effects are negative, irrespective of original sign.

In [25]:
PT_contrasts_original_df = pd.read_excel("../output/original_contrasts.xlsx")

In [37]:
PT_contrasts_original_df.tail()

,Effect,Item 1,Item 2,A1,B1,A2,B2,OR,p-value,OR2,Significance,LogOR
8,Isolation Effect,4,11,62,33,31,110,6.666667,3.008508e-11,6.666667,True,1.897120
9,Overweighting of small probabilities,5,6,9,57,48,18,0.059211,4.830963e-12,16.888889,True,-2.826656
10,Overweighting of small probabilities,9,10,61,5,20,46,28.060000,6.049779e-14,28.060000,True,3.334345
11,Framing Effect,12,13,11,59,47,21,0.083303,1.728730e-10,12.004329,True,-2.485267
12,Range Adaptation,14,15,12,56,45,19,0.090476,9.017691e-10,11.052632,True,-2.402669


### Split the contrast column in lor_df to two separate columns (one for each item)

In [24]:
lodds_df["Item 1"] = lodds_df["Contrasts"].str.split(" vs ", expand=True)[0]
lodds_df["Item 2"] = lodds_df["Contrasts"].str.split(" vs ", expand=True)[1]

In [42]:
lodds_df["Item 1"] = lodds_df["Item 1"].astype(int)
lodds_df["Item 2"] = lodds_df["Item 2"].astype(int)

### Compute the difference in effect size for the central estimates

In [44]:
lodds_df["LogOR_difference"] = np.nan
for a, b in zip(PT_contrasts_original_df["Item 1"], PT_contrasts_original_df["Item 2"]):
    if np.sign(PT_contrasts_original_df.loc[(PT_contrasts_original_df["Item 1"]==a) & (PT_contrasts_original_df["Item 2"]==b),
                         "LogOR"]).values[0] == 1: # If the original log-odds are positive
        lodds_df.loc[(lodds_df["Item 1"]==a) & (lodds_df["Item 2"]==b),
                             "LogOR_difference"] = lodds_df.loc[
            (lodds_df["Item 1"]==a) & (lodds_df["Item 2"]==b),
                             "lodds"] - PT_contrasts_original_df.loc[
            (PT_contrasts_original_df["Item 1"]==a) & (PT_contrasts_original_df["Item 2"]==b),
                             "LogOR"].values[0] # Compute the difference between meta analytic effects and original
    else: # If the original log-odds are negative
            lodds_df.loc[(lodds_df["Item 1"]==a) & (lodds_df["Item 2"]==b),
                             "LogOR_difference"] =     (lodds_df.loc[
            (lodds_df["Item 1"]==a) & (lodds_df["Item 2"]==b),
                             "lodds"] - PT_contrasts_original_df.loc[
            (PT_contrasts_original_df["Item 1"]==a) & (PT_contrasts_original_df["Item 2"]==b),
                             "LogOR"].values[0]) * -1 # Compute the difference between meta analytic effects and original and multiply by negative 1

### Compute the difference in effect size for the upper bounds

In [46]:
lodds_df["ub_difference"] = np.nan
for a, b in zip(PT_contrasts_original_df["Item 1"], PT_contrasts_original_df["Item 2"]):
    if np.sign(PT_contrasts_original_df.loc[(PT_contrasts_original_df["Item 1"]==a) & (PT_contrasts_original_df["Item 2"]==b),
                         "LogOR"]).values[0] == 1: # If the original log-odds are positive
        lodds_df.loc[(lodds_df["Item 1"]==a) & (lodds_df["Item 2"]==b),
                             "ub_difference"] =     lodds_df.loc[
            (lodds_df["Item 1"]==a) & (lodds_df["Item 2"]==b),
                             "ub"] - PT_contrasts_original_df.loc[
            (PT_contrasts_original_df["Item 1"]==a) & (PT_contrasts_original_df["Item 2"]==b),
                             "LogOR"].values[0] # Compute the difference between meta analytic effects and original
    else: # If the original log-odds are negative
            lodds_df.loc[(lodds_df["Item 1"]==a) & (lodds_df["Item 2"]==b),
                             "ub_difference"] =     (lodds_df.loc[
            (lodds_df["Item 1"]==a) & (lodds_df["Item 2"]==b),
                             "ub"] - PT_contrasts_original_df.loc[
            (PT_contrasts_original_df["Item 1"]==a) & (PT_contrasts_original_df["Item 2"]==b),
                             "LogOR"].values[0]) * -1 # Compute the difference between meta analytic effects and original and multiply by negative 1

### Compute the difference in effect size for the lower bounds

In [47]:
lodds_df["lb_difference"] = np.nan
for a, b in zip(PT_contrasts_original_df["Item 1"], PT_contrasts_original_df["Item 2"]):
    if np.sign(PT_contrasts_original_df.loc[(PT_contrasts_original_df["Item 1"]==a) & (PT_contrasts_original_df["Item 2"]==b),
                         "LogOR"]).values[0] == 1: # If the original log-odds are positive
        lodds_df.loc[(lodds_df["Item 1"]==a) & (lodds_df["Item 2"]==b),
                             "lb_difference"] =     lodds_df.loc[
            (lodds_df["Item 1"]==a) & (lodds_df["Item 2"]==b),
                             "lb"] - PT_contrasts_original_df.loc[
            (PT_contrasts_original_df["Item 1"]==a) & (PT_contrasts_original_df["Item 2"]==b),
                             "LogOR"].values[0] # Compute the difference between meta analytic effects and original
    else: # If the original log-odds are negative
            lodds_df.loc[(lodds_df["Item 1"]==a) & (lodds_df["Item 2"]==b),
                             "lb_difference"] =     (lodds_df.loc[
            (lodds_df["Item 1"]==a) & (lodds_df["Item 2"]==b),
                             "lb"] - PT_contrasts_original_df.loc[
            (PT_contrasts_original_df["Item 1"]==a) & (PT_contrasts_original_df["Item 2"]==b),
                             "LogOR"].values[0]) * -1 # Compute the difference between meta analytic effects and original and multiply by negative 1

### Assign effect names to the meta-analytic countrasts

In [51]:
for a, b in zip(PT_contrasts_original_df["Item 1"], PT_contrasts_original_df["Item 2"]):
    lodds_df.loc[(lodds_df["Item 1"]==a) & (lodds_df["Item 2"]==b),
                             "Effect"] =  PT_contrasts_original_df.loc[
            (PT_contrasts_original_df["Item 1"]==a) & (PT_contrasts_original_df["Item 2"]==b),
                             "Effect"].values[0]

In [52]:
lodds_df

,lodds,lb,ub,p-value,Contrasts,Succesful Replication,Item 1,Item 2,LogOR_difference,ub_difference,lb_difference,Effect
0,-1.581017,-1.720487,-1.441547,2.305722e-109,1 vs 2,Yes,1,2,-1.541009,-1.680479,-1.401539,Certainty Effect
1,-1.969901,-2.081812,-1.857990,7.994243e-261,3 vs 4,Yes,3,4,-0.047020,-0.158931,0.064891,Certainty Effect
2,1.332841,1.235518,1.430164,1.052629e-158,7 vs 8,Yes,7,8,-1.372080,-1.274757,-1.469403,Certainty Effect
3,-3.332428,-3.565245,-3.099612,3.568163e-173,3 vs 7,Yes,3,7,-0.440333,-0.673149,-0.207516,Reflection Effect
4,0.026007,-0.143975,0.195989,7.642756e-01,4 vs 8,No,4,8,-0.923074,-0.753092,-1.093056,Reflection Effect
5,-3.532676,-3.845517,-3.219836,1.540525e-108,5 vs 9,Yes,5,9,-0.814586,-1.127427,-0.501746,Reflection Effect
6,0.918727,0.783602,1.053853,1.636367e-40,6 vs 10,Yes,6,10,-0.895011,-0.759885,-1.030137,Reflection Effect
7,0.582853,0.349460,0.816246,9.849195e-07,16 vs 17,Yes,16,17,-1.982096,-1.748703,-2.215489,Reflection Effect
8,1.703716,1.598432,1.809000,9.248932e-221,4 vs 11,Yes,4,11,-0.193404,-0.088120,-0.298688,Isolation Effect
9,-2.782012,-2.943432,-2.620591,4.002685e-250,5 vs 6,Yes,5,6,-0.044644,-0.206065,0.116777,Overweighting of small probabilities


In [53]:
lodds_df.to_excel("../output/Meta analysis log odds.xlsx", index=False)
lodds_df.to_csv("../output/Meta analysis log odds.csv", index=False)